In [1]:
import numpy as np 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import xgboost as xgb
from scipy.stats import uniform,randint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV,train_test_split,RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from imblearn.under_sampling import ClusterCentroids, TomekLinks,RandomUnderSampler
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import f_classif,chi2
from sklearn.preprocessing import Binarizer, scale
from sklearn.metrics import roc_auc_score

import random
random.seed(1301) 

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
path = r'/gdrive/My Drive/KAEMH/newyear_newyou/dataset_allfeatures.csv'
train_data=pd.read_csv(path,index_col=0)

In [0]:
#preparing the data for train_test_split

x=train_data.iloc[:,:-1]
y=train_data.iloc[:,-1]

In [0]:
#applying the normalization with PCA

x_normalized = normalize(x, axis=0)
pca = PCA(n_components=2)
x_pca = pca.fit_transform(x_normalized)
x['PCA1'] = x_pca[:,0]
x['PCA2'] = x_pca[:,1]


In [6]:
def identify_constant_features(dataframe):
    count_uniques = dataframe.apply(lambda x: len(x.unique()))
    constants = count_uniques[count_uniques == 1].index.tolist()
    return constants

constant_features_test = set(identify_constant_features(x))

print('There were {} constant features in dataset.'.format(len(constant_features_test)))
print(constant_features_test)

There were 17 constant features in dataset.
{'Record.Type.DescOther.Organization', 'IQ', 'AdmitType', 'Record.Type.DescReligious.Organization', 'Record.Type.DescStudent', 'Record.Type.DescGovernment.Organization', 'Record.Type.DescAlumnus.a..Non.Degreed', 'Record.Type.DescCorporation', 'Record.Type.DescEstate.of.Alumnus.a', 'Record.Type.DescDonor.Advised.Fund', 'Record.Type.DescFoundation', 'Record.Type.DescFundraising.Consortium', 'Record.Type.DescEstate.of.Thunderbird.Alumus.a', 'Record.Type.DescPersonal.or.Family.Foundation', 'Record.Type.DescASU.Affiliate', 'Record.Type.DescCorporate.Foundation', 'Record.Type.DescEstate'}


In [0]:
# Drop the constant features

x.drop(constant_features_test, inplace=True, axis=1)

In [0]:
#Scaling the data and preparing for feature selection

#p=86 Auc score overall : 0.8612
#p=86 Auc score with Undersamplimg : 0.8840
#p=75 Auc score with Undersamplimg : 0.8886
#p=60  #AUC score with Undersampling : 0.8920
#p=70 AUC score with undersampling : 0.8868
p=60

x_bin = Binarizer().fit_transform(scale(x))
selectChi2 = SelectPercentile(chi2, percentile=p).fit(x_bin, y)
selectF_classif = SelectPercentile(f_classif, percentile=p).fit(x, y)

In [9]:
#features selection using chi2

chi2_selected = selectChi2.get_support()
chi2_selected_features = [ f for i,f in enumerate(x.columns) if chi2_selected[i]]
print('Chi2 selected {} features {}.'.format(chi2_selected.sum(),chi2_selected_features))

Chi2 selected 398 features ['Alumni_Id', 'Age', 'Gender.M', 'Local', 'Pref.Class.Year', 'Ethnic.Desc', 'Ethnic.DescAmerican.Indian.Alaska.Native', 'Ethnic.DescRace.and.Ethnicity.Unknown', 'Ethnic.DescTwo.or.More.Races', 'Record.Type.DescFaculty.Staff', 'Record.Type.DescParent', 'Record.Type.DescPast.Faculty.Staff', 'Record.Type.DescPast.Parent', 'Affiliated', 'Affil.Code.DescAA.Alumni.Ambassadors', 'Affil.Code.DescAA.Starbucks.CAP', 'Affil.Code.DescOther', 'MemberCurrent.Member', 'Member.LevelGold', 'Member.LevelMaroon', 'Member.LevelSparky', 'Activity.Participation.DescInvited', 'Activity.Participation.DescNo.Show', 'User.Group.DescASU.Enterprise.Marketing.Hub', 'User.Group.DescCollege.of.Health.Solutions', 'User.Group.DescFDN.IRM', 'User.Group.DescFDN.Public.Events', 'User.Group.DescZ..CONVERSION', 'CoursePerf', 'NUM_UNDERGRAD_DEGREES', 'NUM_GRAD_DEGREES', 'ACADCAREERLENGTH', 'TOTALSTUDENTWORKERHRS', 'AVGSTUDENTWORKERHRSDAILY', 'NUMDAYSWORKED', 'SWDURATION', 'SWHRDENSITY', 'NUMADVAPP

In [10]:
#features selection using f_classif

f_classif_selected = selectF_classif.get_support()
f_classif_selected_features = [ f for i,f in enumerate(x.columns) if f_classif_selected[i]]
print('F_classif selected {} features {}.'.format(f_classif_selected.sum(),f_classif_selected_features))

F_classif selected 398 features ['Alumni_Id', 'Age', 'Gender.M', 'Local', 'Pref.Class.Year', 'Ethnic.Desc', 'Ethnic.DescAmerican.Indian.Alaska.Native', 'Ethnic.DescRace.and.Ethnicity.Unknown', 'Ethnic.DescTwo.or.More.Races', 'Record.Type.DescAlumnus.a..Thunderbird', 'Record.Type.DescFaculty.Staff', 'Record.Type.DescParent', 'Record.Type.DescPast.Faculty.Staff', 'Record.Type.DescPast.Parent', 'Affiliated', 'Affil.Code.DescAA.All.Sts.Cath.Newman.Cntr.Chaptr', 'Affil.Code.DescAA.Alumni.Ambassadors', 'Affil.Code.DescAA.Starbucks.CAP', 'Affil.Code.DescAA.SunDevils..Advocacy.Network', 'Affil.Code.DescOther', 'MemberCurrent.Member', 'Member.LevelGold', 'Member.LevelMaroon', 'Member.LevelSparky', 'Activity.Participation.DescInvited', 'Activity.Participation.DescNo.Show', 'User.Group.DescASU.Enterprise.Marketing.Hub', 'User.Group.DescCollege.of.Health.Solutions', 'User.Group.DescFDN.Public.Events', 'User.Group.DescZ..CONVERSION', 'CoursePerf', 'NUM_UNDERGRAD_DEGREES', 'NUM_GRAD_DEGREES', 'ACADC

In [11]:
#common features

selected = chi2_selected & f_classif_selected
print('Chi2 & F_classif selected {} features'.format(selected.sum()))
features = [ f for f,s in zip(x.columns, selected) if s]
print (features)

Chi2 & F_classif selected 301 features
['Alumni_Id', 'Age', 'Gender.M', 'Local', 'Pref.Class.Year', 'Ethnic.Desc', 'Ethnic.DescAmerican.Indian.Alaska.Native', 'Ethnic.DescRace.and.Ethnicity.Unknown', 'Ethnic.DescTwo.or.More.Races', 'Record.Type.DescFaculty.Staff', 'Record.Type.DescParent', 'Record.Type.DescPast.Faculty.Staff', 'Record.Type.DescPast.Parent', 'Affiliated', 'Affil.Code.DescAA.Alumni.Ambassadors', 'Affil.Code.DescAA.Starbucks.CAP', 'Affil.Code.DescOther', 'MemberCurrent.Member', 'Member.LevelGold', 'Member.LevelMaroon', 'Member.LevelSparky', 'Activity.Participation.DescInvited', 'Activity.Participation.DescNo.Show', 'User.Group.DescASU.Enterprise.Marketing.Hub', 'User.Group.DescCollege.of.Health.Solutions', 'User.Group.DescFDN.Public.Events', 'User.Group.DescZ..CONVERSION', 'CoursePerf', 'NUM_UNDERGRAD_DEGREES', 'NUM_GRAD_DEGREES', 'ACADCAREERLENGTH', 'TOTALSTUDENTWORKERHRS', 'AVGSTUDENTWORKERHRSDAILY', 'NUMDAYSWORKED', 'SWDURATION', 'SWHRDENSITY', 'NUMADVAPPOINTMENTS', 'A

In [0]:
#selecting the final features and splitting the data

x_feature=x[features]


x_train, x_test, y_train, y_test = train_test_split(x_feature.values, y.values, random_state=1301, stratify=y, test_size=0.25)


In [0]:
ratio=float(np.sum(y_train == 0) )/ np.sum(y_train==1)
print(ratio)

3464.9532710280373


In [0]:
#using XGboost with actual train and test

xgb_model = xgb.XGBClassifier(n_estimators=1000,learning_rate=0.1, seed=1301,subsample=1.0,reg_alpha=0.03, gamma=0.3,max_depth=5,is_unbalance=False,num_leaves=4,scale_pos_weight=ratio,colsample_bytree=0.9,n_thread=4)
xgb_model.fit(x_train, y_train,early_stopping_rounds=50,eval_metric="auc",eval_set=[(x_train,y_train),(x_test,y_test)])



[0]	validation_0-auc:0.918689	validation_1-auc:0.705161
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 50 rounds.
[1]	validation_0-auc:0.933218	validation_1-auc:0.756217
[2]	validation_0-auc:0.955125	validation_1-auc:0.749905
[3]	validation_0-auc:0.958304	validation_1-auc:0.85354
[4]	validation_0-auc:0.970935	validation_1-auc:0.852674
[5]	validation_0-auc:0.976286	validation_1-auc:0.870195
[6]	validation_0-auc:0.977401	validation_1-auc:0.877694
[7]	validation_0-auc:0.979553	validation_1-auc:0.895643
[8]	validation_0-auc:0.986303	validation_1-auc:0.89443
[9]	validation_0-auc:0.988355	validation_1-auc:0.896149
[10]	validation_0-auc:0.989723	validation_1-auc:0.894846
[11]	validation_0-auc:0.990116	validation_1-auc:0.895605
[12]	validation_0-auc:0.990689	validation_1-auc:0.891371
[13]	validation_0-auc:0.991495	validation_1-auc:0.890448
[14]	validation_0-auc:0.992127	validation_1-auc:0.892225


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0.3,
              is_unbalance=False, learning_rate=0.1, max_delta_step=0,
              max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
              n_jobs=1, n_thread=4, nthread=None, num_leaves=4,
              objective='binary:logistic', random_state=0, reg_alpha=0.03,
              reg_lambda=1, scale_pos_weight=3464.9532710280373, seed=1301,
              silent=None, subsample=1.0, verbosity=1)

In [0]:
print('Overall AUC:', roc_auc_score(y_test, xgb_model.predict_proba(x_test, ntree_limit=xgb_model.best_iteration)[:,1]))

Overall AUC: 0.8746552829523413


In [0]:
y_pred_train=pd.DataFrame(xgb_model.predict(x_train),columns=["TARGET"])

y_pred_train.TARGET.value_counts()

print(classification_report(y_train,y_pred_train))
print(confusion_matrix(y_train,y_pred_train))


              precision    recall  f1-score   support

         0.0       1.00      0.98      0.99    296599
         1.0       0.01      1.00      0.02        86

    accuracy                           0.98    296685
   macro avg       0.51      0.99      0.51    296685
weighted avg       1.00      0.98      0.99    296685

[[289570   7029]
 [     0     86]]


In [0]:
y_pred = xgb_model.predict(x_test,ntree_limit=xgb_model.best_iteration)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))


NameError: ignored

In [0]:
auc = roc_auc_score(y_test, xgb_model.predict(x_test))
print("Performance : ", auc)

Performance :  0.7424406242020336


In [13]:
#using XGboost with Random Under sampling
rus = RandomUnderSampler(return_indices=True,ratio=0.25)
x_rus, y_rus, id_rus = rus.fit_sample(x_train, y_train)

print('Removed indexes:', id_rus)


Removed indexes: [ 53480  68260 220179 199416 117821 285506 287599 116769 232641  99600
 186911  15397  17102 122547 348511 193945 189995  60393 213193  69319
 142697   4986 356394 318264 301595 318553 180453   3915  96356 105526
 305206 110436 109636 246838  99838 145073  51769 201295  41414 304890
 306634  48009  62239 360775 329960   6032 345241  21216 347405  42129
   9175 124316 332693  86341 323342  76851 187733 198743  59808 146030
 139300 127752 325234 362739 204845 283180  78555  29021   6159 109012
 280608 131327 248704 250108  75901 328105 143502  86954 304947  96540
 191485  53988 138339 336004  31289  39580 312310 336708  83589 332005
 214102 230376  33002 299935 303053 163534 191346 196578  82699 117383
 345289 331219 106372 125377 199812 251650  10030  41536  73341  44461
 177059   1691 183495 122687  72041  23467 189633  72759 223345 184936
 104708 201839 135364 264496 107697 132768 216573  24118  56555 215226
 152647 265020 283260 262266 321105 331115  89111  73926 277

In [14]:
print(len(y_rus[y_rus==0]))
print(len(y_rus[y_rus==1]))

428
107


In [15]:
#using XGboost 

xgb_model = xgb.XGBClassifier(n_estimators=1000,learning_rate=0.1, seed=1301,subsample=1.0,reg_alpha=0.03, gamma=0.3,max_depth=5,is_unbalance=False,num_leaves=31,scale_pos_weight=4,colsample_bytree=0.5,n_thread=4)
xgb_model.fit(x_rus, y_rus,early_stopping_rounds=50,eval_metric="auc",eval_set=[(x_rus,y_rus),(x_test,y_test)])



[0]	validation_0-auc:0.927319	validation_1-auc:0.748436
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 50 rounds.
[1]	validation_0-auc:0.970882	validation_1-auc:0.800454
[2]	validation_0-auc:0.973251	validation_1-auc:0.815107
[3]	validation_0-auc:0.974703	validation_1-auc:0.836057
[4]	validation_0-auc:0.982171	validation_1-auc:0.847853
[5]	validation_0-auc:0.981308	validation_1-auc:0.850242
[6]	validation_0-auc:0.98549	validation_1-auc:0.864216
[7]	validation_0-auc:0.986298	validation_1-auc:0.862597
[8]	validation_0-auc:0.987586	validation_1-auc:0.858246
[9]	validation_0-auc:0.98787	validation_1-auc:0.863822
[10]	validation_0-auc:0.989704	validation_1-auc:0.857674
[11]	validation_0-auc:0.989486	validation_1-auc:0.85619
[12]	validation_0-auc:0.993045	validation_1-auc:0.859365
[13]	validation_0-auc:0.995273	validation_1-auc:0.865596
[14]	validation_0-auc:0.996637	validation_1-auc:0.868184
[

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.3,
              is_unbalance=False, learning_rate=0.1, max_delta_step=0,
              max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
              n_jobs=1, n_thread=4, nthread=None, num_leaves=31,
              objective='binary:logistic', random_state=0, reg_alpha=0.03,
              reg_lambda=1, scale_pos_weight=4, seed=1301, silent=None,
              subsample=1.0, verbosity=1)

In [16]:
print('Overall AUC:', roc_auc_score(y_test, xgb_model.predict_proba(x_test, ntree_limit=xgb_model.best_iteration)[:,1]))

Overall AUC: 0.8790228249660371


In [17]:
y_pred_train=pd.DataFrame(xgb_model.predict(x_rus),columns=["TARGET"])

y_pred_train.TARGET.value_counts()

print(classification_report(y_rus,y_pred_train))
print(confusion_matrix(y_rus,y_pred_train))


              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       428
         1.0       0.99      0.99      0.99       107

    accuracy                           1.00       535
   macro avg       0.99      0.99      0.99       535
weighted avg       1.00      1.00      1.00       535

[[427   1]
 [  1 106]]


In [18]:
y_pred = xgb_model.predict(x_test,ntree_limit=xgb_model.best_iteration)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))


[[113638   9945]
 [    12     24]]
              precision    recall  f1-score   support

         0.0       1.00      0.92      0.96    123583
         1.0       0.00      0.67      0.00        36

    accuracy                           0.92    123619
   macro avg       0.50      0.79      0.48    123619
weighted avg       1.00      0.92      0.96    123619



In [19]:
auc = roc_auc_score(y_test, xgb_model.predict(x_test))
print("Performance : ", auc)

Performance :  0.7932792805914514


In [31]:
y_pred_fullset = xgb_model.predict(x_feature.values,ntree_limit=xgb_model.best_iteration)

print(confusion_matrix(y, y_pred_fullset))
print(classification_report(y,y_pred_fullset))


[[454675  39658]
 [    13    130]]
              precision    recall  f1-score   support

         0.0       1.00      0.92      0.96    494333
         1.0       0.00      0.91      0.01       143

    accuracy                           0.92    494476
   macro avg       0.50      0.91      0.48    494476
weighted avg       1.00      0.92      0.96    494476



In [0]:
y_pred_proba = xgb_model.predict_proba(x_feature.values,ntree_limit=xgb_model.best_iteration)



In [24]:
print(y_pred_proba)

[[0.92569304 0.07430694]
 [0.8780515  0.12194846]
 [0.8966347  0.10336529]
 ...
 [0.7074418  0.2925582 ]
 [0.71518946 0.28481057]
 [0.46451426 0.53548574]]


In [0]:
#prediction file

pred=pd.DataFrame(y_pred_fullset,columns=["TARGET"])

pd.concat([x['Alumni_Id'],pred['TARGET']],axis=1).to_csv('/gdrive/My Drive/KAEMH/newyear_newyou/EmailList_predictions.csv', index = None)


In [0]:
#prediction file with score

pred=pd.DataFrame(y_pred_proba,columns=["TARGET_0","TARGET_1"])

pd.concat([x['Alumni_Id'],pred["TARGET_0"],pred["TARGET_1"]],axis=1).to_csv('/gdrive/My Drive/KAEMH/newyear_newyou/EmailList_score.csv', index = None)
